In [5]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import minimize, basinhopping


#Step 1: Get the data (gotta convert Matlab code into python)

def later_getData(arg1, arg2, param):
    # Placeholder function to simulate data retrieval
    # Let's assume it returns a list where the first element is reaction times (RTs)
    RTs = np.random.normal(loc=0.5, scale=0.1, size=100)  # Example data: 100 random RTs
    return [RTs]
# Retrieve the data (similar to later_getData in MATLAB)
data = later_getData([], [], 0.2)

# Extract RTs
RTs = data[0]

del data

#Step 2: Define the objective function

#parameters of LATER Model
muR = np.mean(RTs)
delta_s = np.std(RTs, ddof = 1)
mu = muR/delta_s
sigma = 1/delta_s

# Assuming 'RTs' is a list or numpy array of reaction times (RTs),
# and 'fits' is a 2-element vector of [muR, deltaS] (model parameters)
def laterErrFcn(fits, RTs):
    muR, delta_s = fits

    # Compute likelihood for each RT (assuming Gaussian likelihood for simplicity)
    likelihoods = (1 / (delta_s * np.sqrt(2 * np.pi))) * np.exp(-((RTs - muR)**2) / (2 * delta_s**2))

    # Take the logarithm of the likelihood
    log_likelihoods = np.log(likelihoods)

    # Sum the log-likelihoods
    total_log_likelihood = np.sum(log_likelihoods)

    # Return the negative of the total log-likelihood (since we want to minimize this function)
    return -total_log_likelihood


#Step 3: Definining initial conditions
#Define the bounds for the parameters (muR and deltaS)
lowerBounds = [0.001, 0.001]
upperBounds = [1000, 1000]
bounds = [(low, high) for low, high in zip(lowerBounds, upperBounds)]

# Initial values (pick based on empirical statistics of RT distribution)
initialValues = [0.5, 0.1]  # Example initial values for muR and deltaS

# Perform the minimization using the L-BFGS-B algorithm with bounds
result = minimize(laterErrFcn, initialValues, args=(RTs,), bounds=bounds, method='L-BFGS-B')

#Step 4: Run the fits
# Define the minimization problem
def objective_function(fits):
    return laterErrFcn(fits, RTs)

# Create the 'minimizer' function for local minimization
def local_minimizer(fits):
    result = minimize(objective_function, fits, bounds=bounds, method='L-BFGS-B', options={'maxiter': 3000})
    return result

# Set up the basinhopping global optimization
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}

# Perform the global optimization using basinhopping
result = basinhopping(objective_function, initialValues, minimizer_kwargs=minimizer_kwargs, niter=100)

# Get the best-fitting parameter values and the negative log-likelihood
fits = result.x  # Optimal parameter values
nllk = result.fun  # Value of the objective function (negative log-likelihood)

print(f"Best fitting parameters: {fits}")
print(f"Negative log likelihood given by objective function: {nllk}")

#Part 5: Evaluate fits
print("Typically you can do further statistical analysis or plot the fitted model with the data to determine whether the fit is reasonable")


<ipython-input-5-641b3ccd2a5a>:38: RuntimeWarning: divide by zero encountered in log
  log_likelihoods = np.log(likelihoods)


Best fitting parameters: [0.49801886 0.10065717]
Negative log likelihood given by objective function: -87.70964234966407
